# Skin Disease Image Classification Project

This notebook presents a deep learning approach for **classifying skin diseases from images**. The key objectives of this project are to build an accurate and efficient model while managing computational resources and training time.

## Dataset and Classes
The original dataset contained multiple classes of skin diseases. To **reduce training time and complexity**, we have **merged/reduced the number of classes**. This helps the model learn more generalized patterns, avoids overfitting on underrepresented classes, and speeds up experimentation.

## Preprocessing
- Image resizing and normalization to standardize inputs.
- Data augmentation (like rotations, flips, and color adjustments) to improve model generalization.
- Splitting into training, validation, and test sets to evaluate performance properly.

## Model and Hyperparameter Tuning
We have implemented a deep learning classifier using convolutional neural networks (CNNs). To **optimize model performance**, we performed **hyperparameter tuning using Optuna**:
- Optuna efficiently searches for the best hyperparameters by exploring different combinations and pruning underperforming trials.
- Parameters tuned include learning rate, batch size, optimizer type, dropout rates, and number of layers/neurons.
- The **best parameters** were selected based on validation performance to improve accuracy while reducing overfitting.

## Key Decisions
1. **Class Reduction:** Reduced the number of classes to manage training time and computational cost, while maintaining clinically meaningful categories.
2. **Optuna Hyperparameter Tuning:** Allowed efficient identification of the best hyperparameters without exhaustive search.
3. **Data Augmentation:** Improved generalization and model robustness on unseen data.

This setup ensures a balance between **model performance, training efficiency, and interpretability**, making it suitable for practical deployment in skin disease detection workflows.


This notebook presents a deep learning approach for classifying skin diseases from images. The main objective of this project is to build an accurate and efficient model while managing computational resources and training time.

The original dataset contained multiple classes of skin diseases. To reduce complexity and improve training efficiency, the number of classes was reduced. This decision allowed the model to focus on learning generalized patterns and avoid overfitting on underrepresented classes.

The model was trained multiple times using Kaggle, and the process was time-consuming. Initial attempts yielded a low F1 score of 0.23. To improve performance, **Optuna** was used for hyperparameter tuning, optimizing parameters such as learning rate, batch size, optimizer type, dropout rates, and network architecture. After multiple training cycles and fine-tuning, the F1 score improved significantly from **0.23 to 0.70**.

Although the current model achieves a respectable F1 score of 0.70, there is room for further improvement. In future iterations, I plan to build a more advanced model and continue hyperparameter tuning, with the goal of increasing the F1 score to **0.85–0.90**. This will involve experimenting with different architectures, fine-tuning training strategies, and systematically optimizing model parameters to achieve higher accuracy and reliability.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import kagglehub
sd20co001_image_dataset_for_skindiseases_dry_oily_normalskin_path = kagglehub.dataset_download('sd20co001/image-dataset-for-skindiseases-dry-oily-normalskin')

print("Data source import complete.")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sd20co001/image-dataset-for-skindiseases-dry-oily-normalskin")

print("Path to dataset files:", path)

In [ ]:
!pip install optuna -q
!pip install wandb -q

In [ ]:
import torch
import os
import cv2
from pathlib import Path
from torchvision import transforms
from torchvision.transforms import v2
from PIL import Image
import pandas as pd
from torch.utils.data import DataLoader , Dataset , random_split , WeightedRandomSampler
from sklearn.model_selection import train_test_split
import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix
import torch.optim as optim
from tqdm import tqdm # progress bars
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
from torchvision import models
import timm
import math
from ast import Pass
import time
import copy
from collections import Counter
from optuna.pruners import MedianPruner # early stopping for bad trials.
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR, ReduceLROnPlateau
from sklearn.metrics import f1_score
import optuna # hyperparameter tuning engine
from kaggle_secrets import UserSecretsClient
import wandb
import csv

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
ROOT_DIR = "/kaggle/input/image-dataset-for-skindiseases-dry-oily-normalskin"


image_paths = glob.glob(ROOT_DIR + "/**/*.jpg",recursive=True)
image_paths += glob.glob(ROOT_DIR + "/**/*.jpeg",recursive=True)
image_paths += glob.glob(ROOT_DIR + "/**/*.png",recursive=True)
df = pd.DataFrame({
    "image_path" : image_paths,
    "label" : [os.path.basename(os.path.dirname(p)) for p in image_paths]
})
print("Total images:", len(df))
df.sample(10)

In [ ]:
df["label"].value_counts()

In [ ]:
keep_labels = [
    "Acne",
    "Nail Fungus And Other Nail Disease",
    "Vitiligo",
    "Rosacea",
    "Rashes",
    "Shingles",
    "Skin_Cancer_Malignant",
    "Psoriasis_Pictures_Lichen_Planus_And_Related_Diseases",
    "Fungal_Infections_Ringworm_Candidiasis",
    "Atopic_Dermatitis",
    "Eczema",
    "Herpes",
    "Warts",
    "Dermatitis",
    "Actinic keratosis",
    "Benign_Tumors_Keratoses",
    "Chickenpox",
    "Dermatofibroma",
    "Dry_Skin",
    "Hidradenitis-Suppurativa",
    "Lupus_And_Other_Connective_Tissue_Diseases",
    "Oily_Skin",
    "Urticaria_Hives",
    "Light_Diseases_And_Disorders_Of_Pigmentation",
    "Bullous_Disease_Photos",
    "Hair_Loss_Photos_Alopecia_And_Other_Hair_Diseases",
    "Poison_Ivy_Photos_And_Other_Contact_Dermatitis",
    "Other diseases",
    "Pa_Cutaneous_Larva_Migrans",
    "Exanthems_And_Drug_Eruptions"
]

# Filter dataset
df = df[df['label'].isin(keep_labels)].reset_index(drop=True)
df.to_csv("filtered_dataset.csv", index=False)
df.sample(10)

In [ ]:
df["label"].value_counts()

In [ ]:
Label_Map = {
    "Cellulitis_Impetigo_And_Other_Bacterial_Infections": "Bacterial_Infections_Cellulitis_Impetigo",
    "Ba_Cellulitis": "Bacterial_Infections_Cellulitis_Impetigo",
    "Ba_Impetigo": "Bacterial_Infections_Cellulitis_Impetigo",

    "Ringworm": "Fungal_Infections_Ringworm_Candidiasis",
    "Athlete_Foot": "Fungal_Infections_Ringworm_Candidiasis",
    "Tinea_Ringworm_Candidiasis_And_Other_Fungal_Infections": "Fungal_Infections_Ringworm_Candidiasis",

    "Seborrheic_Keratoses_And_Other_Benign_Tumors": "Benign_Tumors_Keratoses",
    "Benign_Tumors": "Benign_Tumors_Keratoses",
    "Benign_Keratosis": "Benign_Tumors_Keratoses",

    "Malignant_Tumors": "Skin_Cancer_Malignant",
    "Malignant_Lesions": "Skin_Cancer_Malignant",
    "Basal_Cell_Carcinoma_and_Other_Carcinoma": "Skin_Cancer_Malignant",
    "Melanoma_Skin_Cancer_Nevi_And_Moles": "Skin_Cancer_Malignant",

    "Actinic_Keratosis": "Actinic_Keratosis_And_Cheilitis",
    "Actinic_Cheilitis": "Actinic_Keratosis_And_Cheilitis",
}
print(f"Lebel_counts_before_marging : {df.label.nunique()}")
df["label"] = df.label.replace(Label_Map)
print(f"Lebel_counts_AFTER_marging : {df.label.nunique()}")

In [ ]:
 # Saving The new CSV file
CSV_PATH = "skin_dataset.csv"
df.to_csv(CSV_PATH, index=False)
print("New_df_saved>>....")

In [ ]:
X = df["image_path"]
y = df["label"]

X_train, X_temp , y_train , y_temp = train_test_split(
    X,
    y,
    test_size = 0.25,
    stratify = y ,
    random_state = 23
)

X_val , X_test , y_val , y_test = train_test_split(
    X_temp ,
    y_temp,
    test_size = 0.5,
    stratify = y_temp,
    random_state = 23
)
print(f"Train ; {X_train.shape} , Test : {X_test.shape} , Val : {X_val.shape}")

In [ ]:
df["label"].unique()

In [ ]:
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

train_transform = v2.Compose([
    v2.Resize((380,380)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ColorJitter(brightness = 0.2,
                  contrast = 0.2,
                  saturation = 0.3),
    v2.RandomVerticalFlip(p = 0.5),
    v2.RandomRotation(25),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(imagenet_mean,
                imagenet_std)
])

val_transform = v2.Compose([
    v2.Resize((380,380)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(imagenet_mean,imagenet_std)
])

test_transform = v2.Compose([
    v2.Resize((380,380)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale= True),
    v2.Normalize(imagenet_mean,
                imagenet_std)
])
print(f"train {train_transform} , val : {val_transform} , test : {test_transform}")

In [ ]:
train_df = pd.DataFrame({
    "image_path" : X_train,
    "label" : y_train
})
val_df = pd.DataFrame({
    "image_path" : X_val,
    "label" : y_val
})
test_df = pd.DataFrame({
    "image_path" : X_test,
    "label" : y_test
})

SAVE_DIR = "/kaggle/working/"

# Create the directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

train_df.to_csv(os.path.join(SAVE_DIR , "train.csv") , index = False)
val_df.to_csv(os.path.join(SAVE_DIR, "val.csv") , index = False)
test_df.to_csv(os.path.join(SAVE_DIR , "test.csv") , index = False)

print(f"Saved train.csv, val.csv, test.csv in {SAVE_DIR}")
print(f"Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")

In [ ]:
all_labels = sorted(train_df["label"].unique())
cls_to_idx = {label : idx for idx , label in enumerate(all_labels)}

class_counts = train_df["label"].value_counts().to_dict()
counts = np.array([class_counts.get(label, 0) for label in all_labels])

# Class-Balanced Loss Based on Effective Number of Samples (CVPR 2019).
beta = 0.9999
effective_num = 1.0 - np.power(beta, counts)
weights = (1.0 - beta) / effective_num
weights = weights / np.sum(weights)

# Convert to tensor for PyTorch
class_weights = torch.tensor(weights, dtype=torch.float32)

sample_weights = [class_weights[cls_to_idx[label]] for label in train_df["label"]]

# diyctionary for reference
normalized_weights = {
    cls: w for cls, w in zip(class_counts.keys(), weights)
}

print("✅ Class-balanced Weights:")
for k, v in normalized_weights.items():
    print(f"{k}: {v:.6f}")

In [ ]:
class SKINDISEASE(Dataset):
    def __init__(self,df, transforms = None , label_dict = None):
        self.data = df
        self.transforms = transforms
        self.label_dict = label_dict
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        image_path = self.data.iloc[idx]["image_path"]
        label = self.data.iloc[idx]["label"]

        if self.label_dict:
            label = self.label_dict[label]
        img = Image.open(image_path).convert("RGB")

        if self.transforms :
            img = self.transforms(img)
        return img , label # WHY ?? int(label) -> Look at the next cell

In [ ]:
train_dataset = SKINDISEASE(
    df = train_df,
    transforms = train_transform,
    label_dict = cls_to_idx
)
val_dataset = SKINDISEASE(
    df = val_df ,
    transforms = val_transform ,
    label_dict = cls_to_idx
)
test_dataset = SKINDISEASE(
    df = test_df,
    transforms = test_transform ,
    label_dict = cls_to_idx
)
display(train_dataset , val_dataset , test_dataset)

In [ ]:
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataloader = DataLoader(
                          dataset = train_dataset,
                          batch_size=32,
                          sampler=sampler,
                          num_workers = 2,
                          pin_memory = True)
val_dataloader = DataLoader(
                          dataset = val_dataset ,
                          batch_size = 32,
                          shuffle = False,
                          num_workers = 2,
                          pin_memory = True
)
test_dataloader = DataLoader(
                          dataset = test_dataset ,
                          batch_size = 32,
                          shuffle = False,
                          num_workers = 2,
                          pin_memory = True
)
print(f"Train_dataloader length : {len(train_dataloader)}  | test_data_loader : {len(val_dataloader)} Test DataLoader length : {len(test_dataloader)}")


In [ ]:
(len(cls_to_idx)) == (train_df["label"].nunique())

In [ ]:
num_classes = len(cls_to_idx)
num_classes

In [ ]:
def get_model(name ,
             num_classes ,
             pretrained = True,
             device = "cuda"):
    if name.startswith("efficientnet_b"):
        if pretrained:
            # Correctly specify the weight enum
            if name == "efficientnet_b2":
                weights = models.EfficientNet_B2_Weights.IMAGENET1K_V1
            # Added similar conditions for other efficientnet_b models if needed
            else:
                 weights = None # Or handle other efficientnet versions

            model = getattr(models , name)(weights=weights)
        else:
            model = getattr(models , name)(weights=None) # Pass weights=None for non-pretrained
        if hasattr(model , "classifier") :
            if isinstance (model.classifier , nn.Sequential) and isinstance(model.classifier[-1] , nn.Linear):
                in_f = model.classifier[-1].in_features
                model.classifier[-1] = nn.Linear(in_f , num_classes)

            else :
                in_f = model.classifier.in_features
                model.classifier = nn.Linear(in_f , num_classes)

    elif name == "densenet121" :
        model = models.densenet121(weights = models.DenseNet121_Weights.IMAGENET1K_V1 if pretrained else None)
        in_f = model.classifier.in_features
        model.classifier = nn.Linear(in_f , num_classes)
    elif name == "resnet50" :
        model = models.resnet50(weights = models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)
        in_f = model.fc.in_features
        model.fc = nn.Linear(in_f , num_classes)
    else:
        raise ValueError(f"Unknown model : {name}")

    return model.to(device)

In [ ]:
def train_one_epoch(model,
                   dataloader,
                   loss_fn,
                   optimizer,
                   device,
                   scheduler=None):
    model.train()
    running_loss = 0.0
    all_preds , all_labels = [] , []

    loop = tqdm(dataloader,desc="Train", leave=False) # Used leave=False to remove the bar after completion
    for images , labels in loop :
        images , labels = images.to(device) , labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()

        if scheduler and isinstance(scheduler,OneCycleLR):
            scheduler.step()

        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(outputs , dim = 1).detach().cpu().numpy()
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.detach().cpu().numpy().tolist())

        loop.set_postfix(loss=loss.item())


    avg_loss = running_loss / len(dataloader.dataset)
    return avg_loss , all_labels , all_preds

def validate(model ,
            dataloader ,
            loss_fn ,
            device):
    model.eval()
    running_loss = 0.0
    all_labels , all_preds = [], []

    with torch.no_grad():
        loop = tqdm(dataloader,desc="Val", leave=False) 
        for images , labels in loop :
            images , labels = images.to(device) , labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs , labels)


            running_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim = 1).detach().cpu().numpy()
            all_preds.extend(preds.tolist())
            all_labels.extend(labels.detach().cpu().numpy().tolist())
            loop.set_postfix(loss=loss.item())


        avg_loss = running_loss / len(dataloader.dataset)
        return avg_loss , all_labels , all_preds

In [ ]:
def get_labels_from_dataset(dataset):
    """
    Try to detect labels list for common dataset types (ImageFolder, custom).
    """

    if hasattr(dataset , "targets"):
        return list(dataset.targets)
    if hasattr(dataset,"labels"):
        return list(dataset.labels)
    if hasattr(dataset,"samples"):
        return [label for _, label in dataset.samples]

    labels = []
    # Assuming the dataset returns (image, label) pairs
    for _, lbl in dataset:
        labels.append(int(lbl))
    return labels

def get_weighted_sampler(dataset):
    labels = get_labels_from_dataset(dataset)
    class_counts = Counter(labels)
    num_samples = len(labels)
    weights = [1.0 / class_counts[lbl] for lbl in labels]
    sampler = WeightedRandomSampler(weights,
                                   num_samples = num_samples,
                                   replacement = True)
    return sampler

def get_macro_f1(y_true, y_pred , num_classes):
    return f1_score(y_true,y_pred,average="macro",labels=list(range(num_classes)))

def load_model(model_path, model_name, num_classes, device):
    """Loads a pre-trained model from a specified path."""
    model = get_model(model_name, num_classes, pretrained=False, device=device)
    # Load the entire checkpoint
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    # Extract the model state dictionary
    model_state_dict = checkpoint.get('model_state', checkpoint) # Used to get with a default for flexibility
    model.load_state_dict(model_state_dict)
    model.eval()
    return model

def predict(model, dataloader, device):
    """Makes predictions on a dataset."""
    model.eval()
    all_preds = []
    with torch.no_grad():
        for images, _ in tqdm(dataloader, desc="Predicting"):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            all_preds.extend(preds.tolist())
    return all_preds

def evaluate(y_true, y_pred, all_labels):
    """Evaluates predictions and displays classification report and confusion matrix."""
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=all_labels))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 10))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=all_labels, yticklabels=all_labels)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
def set_seed(seed):
    """seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
def final_retrain_and_save(best_params, train_dataset, val_dataset, num_classes,
                           class_weights, device, epochs=30, batch_size=32, save_path="best_model.pth"):

    train_sampler = get_weighted_sampler(train_dataset)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler,
                              num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=4, pin_memory=True)

    model = get_model(best_params.get("model_name", "efficientnet_b2"), num_classes=num_classes,
                      pretrained=True, device=device)
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

    opt_name = best_params.get("optimizer", "AdamW")
    lr = best_params.get("lr", 1e-3)
    wd = best_params.get("weight_decay", 1e-4)

    if opt_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    else:
        momentum = best_params.get("momentum", 0.9)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=wd)

    scheduler = None
    sched_name = best_params.get("scheduler", "OneCycleLR")
    if sched_name == "OneCycleLR":
        scheduler = OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(train_loader), epochs=epochs)
    elif sched_name == "CosineAnnealingLR":
        scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    elif sched_name == "ReduceLROnPlateau":
        scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5)

    best_val = 0.0
    for epoch in range(epochs):
        train_loss, _, _ = train_one_epoch(model, train_loader, criterion, optimizer, device, scheduler)
        val_loss, y_true, y_pred = validate(model, val_loader, criterion, device)
        macro_f1 = get_macro_f1(y_true, y_pred, num_classes)
        print(f"[Final Retrain] Epoch {epoch+1}/{epochs} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | F1: {macro_f1:.4f}")

        if macro_f1 > best_val:
            best_val = macro_f1
            torch.save({
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "params": best_params,
                "epoch": epoch,
                "val_macro_f1": best_val
            }, save_path)
            print(f" Saved best model to {save_path} (F1={best_val:.4f})")

    return save_path


In [ ]:
# Best params <- from optuna study [To access the notebook : https://colab.research.google.com/drive/1UcBeG6IA-Zgt8tLCUnwqSTBzIKiuha4a?usp=sharing]
best_params = {
    "model_name" : "efficientnet_b2",
    "optimizer" : "AdamW",
    "lr" : 2.96e-04,
    "weight_decay" : 8.23e-04,
    "scheduler" : None,
    "batch_size" : 16
}

In [ ]:
if __name__ == "__main__":
    train_dataset_full = train_dataloader.dataset
    val_dataset_full = val_dataloader.dataset

    
    # num_classes = len(cls_to_idx)
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
    # print("11")
    # train_df_reduced = get_balanced_subset_df(train_df, n_per_class=120, seed=42)
    # print("train_loaded")
    # val_df_reduced   = get_balanced_subset_df(val_df, n_per_class=30, seed=123)
    # print("val_loaded")
    # train_dataset_reduced = SKINDISEASE(
    #     df = train_df_reduced,
    #     transforms = train_transform,
    #     label_dict = cls_to_idx
    # )
    # val_dataset_reduced = SKINDISEASE(
    #     df = val_df_reduced ,
    #     transforms = val_transform ,
    #     label_dict = cls_to_idx
    # )


    # study = run_optuna_study(train_dataset_reduced, val_dataset_reduced, num_classes, class_weights_tensor,
    #                          device, n_trials=20, epochs=3,
    #                          study_name="skin_hpo_production",
    #                          storage="sqlite:///optuna_skin.db", use_wandb=True)

    # best_params = study.best_trial.params1`
    saved_path = final_retrain_and_save(best_params, train_dataset, val_dataset,
                                        num_classes, class_weights, device,
                                        epochs=15,
                                        batch_size=best_params.get("batch_size", 24),
                                        save_path="best_skin_model.pth")
    print("Final saved model:", saved_path)

In [ ]:
model_path = "/kaggle/working/best_skin_model.pth" 
best_model_name = "efficientnet_b2" 
num_classes = len(cls_to_idx)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_labels = sorted(cls_to_idx.keys()) 

try:
    model = load_model(model_path, best_model_name, num_classes, device)
    print(f"Model loaded successfully from {model_path}")

    y_true_test = [label for _, label in test_dataset]

    y_pred_test = predict(model, test_dataloader, device)

    evaluate(y_true_test, y_pred_test, all_labels)

except FileNotFoundError:
    print(f"Error: Model file not found at {model_path}. Please make sure the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")